### Custom Encoder: Rule-Based

Lightwood uses "Encoders" to convert preprocessed (cleaned) data into **features**. Encoders represent the **feature engineering** step of the data science pipeline; they can either have a set of instructions ("rule-based") or a learned representation (trained on data).

In the following notebook, we will experiment with creating a custom encoder that creates **Label Encoding**. 

For example, imagine we have the following set of categories:

```
MyColumnData = ["apple", "orange", "orange", "banana", "apple", "dragonfruit"]
```

There are 4 categories to consider: "apple", "banana", "orange", and "dragonfruit".

**Label encoding** allows you to refer to these categories as if they were numbers. For example, consider the mapping (arranged alphabetically):

1 - apple <br>
2 - banana <br>
3 - dragonfruit <br>
4 - orange <br>

Using this mapping, we can convert the above data as follows:

```
MyFeatureData = [1, 4, 4, 2, 1, 3]
```

In the following notebook, we will design a **LabelEncoder** for Lightwood for use on categorical data. We will be using the Kaggle "Used Car" [dataset](https://www.kaggle.com/adityadesai13/used-car-dataset-ford-and-mercedes). We've provided a link for you to automatically access this CSV. This dataset describes various details of cars on sale - with the goal of predicting how much this car may sell for.

Let's get started.

In [1]:
import pandas as pd

# Lightwood modules
import lightwood as lw
from lightwood import ProblemDefinition, \
                      JsonAI, \
                      json_ai_from_problem, \
                      code_from_json_ai, \
                      predictor_from_code

INFO:lightwood-2140:No torchvision detected, image helpers not supported.
INFO:lightwood-2140:No torchvision/pillow detected, image encoder not supported


### 1) Load your data

Lightwood works with `pandas.DataFrame`s; load data via pandas as follows:

In [2]:
filename = 'https://raw.githubusercontent.com/mindsdb/benchmarks/main/benchmarks/datasets/used_car_price/data.csv'
df = pd.read_csv(filename)
df.head()

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,A1,2017,12500,Manual,15735,Petrol,150,55.4,1.4
1,A6,2016,16500,Automatic,36203,Diesel,20,64.2,2.0
2,A1,2016,11000,Manual,29946,Petrol,30,55.4,1.4
3,A4,2017,16800,Automatic,25952,Diesel,145,67.3,2.0
4,A3,2019,17300,Manual,1998,Petrol,145,49.6,1.0


We can see a handful of columns above, such as `model, year, price, transmission, mileage, fuelType, tax, mpg, engineSize`. Some columns are numerical whereas others are categorical. We are going to specifically only focus on categorical columns.


### 2) Generate JSON-AI Syntax

We will make a `LabelEncoder` as follows:

(1) Find all unique examples within a column <br>
(2) Order the examples in a consistent way <br>
(3) Label (python-index of 0 as start) each category <br>
(4) Assign the label according to each datapoint. <br>

First, let's generate a JSON-AI syntax so we can automatically identify each column. 

In [3]:
# Create the Problem Definition
pdef = ProblemDefinition.from_dict({
    'target': 'price', # column you want to predict
    #'ignore_features': ['year', 'mileage', 'tax', 'mpg', 'engineSize']
})

# Generate a JSON-AI object
json_ai = json_ai_from_problem(df, problem_definition=pdef)

INFO:lightwood-2140:Dropping features: []
INFO:lightwood-2140:Analyzing a sample of 6920
INFO:lightwood-2140:from a total population of 10668, this is equivalent to 64.9% of your data.
INFO:lightwood-2140:Infering type for: model
INFO:lightwood-2140:Column model has data type categorical
INFO:lightwood-2140:Infering type for: year
INFO:lightwood-2140:Column year has data type integer
INFO:lightwood-2140:Infering type for: price
INFO:lightwood-2140:Column price has data type integer
INFO:lightwood-2140:Infering type for: transmission
INFO:lightwood-2140:Column transmission has data type categorical
INFO:lightwood-2140:Infering type for: mileage
INFO:lightwood-2140:Column mileage has data type integer
INFO:lightwood-2140:Infering type for: fuelType
INFO:lightwood-2140:Column fuelType has data type categorical
INFO:lightwood-2140:Infering type for: tax
INFO:lightwood-2140:Column tax has data type integer
INFO:lightwood-2140:Infering type for: mpg
INFO:lightwood-2140:Column mpg has data ty

Let's take a look at our JSON-AI and print to file.

In [4]:
print(json_ai.to_json())

{
    "features": {
        "model": {
            "encoder": {
                "module": "OneHotEncoder",
                "args": {}
            },
            "data_dtype": "categorical"
        },
        "year": {
            "encoder": {
                "module": "NumericEncoder",
                "args": {}
            },
            "data_dtype": "integer"
        },
        "transmission": {
            "encoder": {
                "module": "OneHotEncoder",
                "args": {}
            },
            "data_dtype": "categorical"
        },
        "mileage": {
            "encoder": {
                "module": "NumericEncoder",
                "args": {}
            },
            "data_dtype": "integer"
        },
        "fuelType": {
            "encoder": {
                "module": "OneHotEncoder",
                "args": {}
            },
            "data_dtype": "categorical"
        },
        "tax": {
            "encoder": {
                "module": "Numeri

### 3) Create your custom encoder (`LabelEncoder`).

Once our JSON-AI is filled, let's make our LabelEncoder. All Lightwood encoders inherit from the `BaseEncoder` class, found [here](https://github.com/mindsdb/lightwood/blob/staging/lightwood/encoder/base.py). 

![BaseEncoder](baseencoder.png)


The `BaseEncoder` has 5 expected calls:

- `__init__`: instantiate the encoder
- `prepare`: Train or create the rules of the encoder
- `encode`: Given data, convert to the featurized representation
- `decode`: Given featurized representations, revert back to data
- `to`: Use CPU/GPU (mostly important for learned representations)

From above, we see that "model", "transmission", and "fuelType" are all categorical columns. These will be the ones we want to modify.

##### `LabelEncoder`

The `LabelEncoder` should satisfy a couple of rules

(1) For the ``__init__`` call: <br>
  - Specify the only argument `is_target`; this asks whether the encoder aims to represent the target column.<br>
  - Set `is_prepared=False` in the initialization. All encoders are prepared using their `prepare()` call, which turns this flag on to `True` if preparation of the encoders is successful. <br>
  - Set `output_size=1`; the output size refers to how many options the represented encoder may adopt. 
    
    
(2) For the ``prepare`` call:
  - Specify the only argument `priming_data`; this provides the `pd.Series` of the data column for the encoder.
  - Find all unique categories in the column data
  - Make a dictionary representing label number to category (reserves 0 as Unknown) and the inverse dictionary
  - Set `is_prepared=True`
  
(3) The `encode()` call will convert each data point's category name into the encoded label.

(4) The `decode()` call will convert a previously encoded label into the original category name.

Given this approach only uses simple dictionaries, there is no need for a dedicated `to()` call (although this would inherit `BaseEncoder`'s implementation).

This implementation would look as follows:

In [5]:
%%writefile LabelEncoder.py

"""
2021.10.13

Create a LabelEncoder that transforms categorical data into a label.
"""
import pandas as pd
import torch

from lightwood.encoder import BaseEncoder
from typing import List, Union
from lightwood.helpers.log import log


class LabelEncoder(BaseEncoder):
    """
    Create a label representation for categorical data. The data will rely on sorted to organize the order of the labels.

    Class Attributes:
    - is_target: Whether this is used to encode the target
    - is_prepared: Whether the encoder rules have been set (after ``prepare`` is called)

    """  # noqa

    is_target: bool
    is_prepared: bool

    is_timeseries_encoder: bool = False
    is_trainable_encoder: bool = False

    def __init__(self, is_target: bool = False) -> None:
        """
        Initialize the Label Encoder

        :param is_target:
        """
        self.is_target = is_target
        self.is_prepared = False

        # Size of the output encoded dimension per data point
        # For LabelEncoder, this is always 1 (1 label per category)
        self.output_size = 1

    # Not all encoders need to be prepared
    def prepare(self, priming_data: pd.Series) -> None:
        """
        Create a LabelEncoder for categorical data.

        LabelDict creates a mapping where each index is associated to a category.

        :param priming_data: Input column data that is categorical.

        :returns: Nothing; prepares encoder rules with `label_dict` and `ilabel_dict`
        """

        # Find all unique categories in the dataset
        categories = priming_data.unique()

        log.info("Categories Detected = " + str(self.output_size))

        # Create the Category labeller
        self.label_dict = {"Unknown": 0}  # Include an unknown category
        self.label_dict.update({cat: idx + 1 for idx, cat in enumerate(categories)})
        self.ilabel_dict = {idx: cat for cat, idx in self.label_dict.items()}

        self.is_prepared = True

    def encode(self, column_data: Union[pd.Series, list]) -> torch.Tensor:
        """
        Convert pre-processed data into the labeled values

        :param column_data: Pandas series to convert into labels
        """
        if isinstance(column_data, pd.Series):
            enc = column_data.apply(lambda x: self.label_dict.get(x, 0)).tolist()
        else:
            enc = [self.label_dict.get(x, 0) for x in column_data]

        return torch.Tensor(enc).int().unsqueeze(1)

    def decode(self, encoded_data: torch.Tensor) -> List[object]:
        """
        Convert torch.Tensor labels into categorical data

        :param encoded_data: Encoded data in the form of a torch.Tensor
        """
        return [self.ilabel_dict[i.item()] for i in encoded_data]


Writing LabelEncoder.py


Some additional notes:
(1) The `encode()` call should be able to intake a list of values, it is optional to make it compatible with `pd.Series` or `pd.DataFrame` <br>
(2) The output of `encode()` must be a torch tensor with dimensionality $N_{rows} x N_{output}$.

Now that the `LabelEncoder` is complete, move this to `~/lightwood_modules` and we're ready to try this out!

In [6]:
from lightwood import load_custom_module
load_custom_module('LabelEncoder.py')

### 4) Edit JSON-AI

Now that we have our `LabelEncoder` script, we have two ways of introducing this encoder:

(1) Change all categorical columns to our encoder of choice <br>
(2) Replace the default encoder (`Categorical.OneHotEncoder`) for categorical data to our encoder of choice <br>

In the first scenario, we may not want to change ALL columns. By switching the encoder on a `Feature` level, Lightwood allows you to control how representations for a given feature are handled. However, suppose you want to replace an approach entirely with your own methodology - Lightwood supports overriding default methods to control how you want to treat a *data type* as well.

Below, we'll show both strategies:

The first strategy requires just specifying which features you'd like to change. Once you have your list, you can manually set the encoder "module" to the class you'd like. **This is best suited for a few columns or if you only want to override a few particular columns as opposed to replacing the `Encoder` behavior for an entire data type**.
#### Strategy 1: Change the encoders for the features directly
```python
for ft in ["model", "transmission", "fuelType"]: # Features you want to replace
    # Set each feature to the custom encoder
    json_ai.features[ft].encoder['module'] = 'LabelEncoder.LabelEncoder'
```


Suppose you have many columns that are categorical- you may want to enforce your approach explicitly without naming each column. This can be done by examining the `data_dtype` of JSON-AI's features. For all features that are type `categorical` (while this is a `str`, it's ideal to import dtype and explicitly check the data type), replace the default `Encoder` with your encoder. In this case, this is `LabelEncoder.LabelEncoder`.
#### Strategy 2: Programatically change *all* encoder assignments for a data type

```python
from lightwood.api import dtype
for i in json_ai.features:
    if json_ai.features[i].data_dtype == dtype.categorical:
        json_ai.features[i].encoder['module'] = 'LabelEncoder.LabelEncoder'
```

We'll go with the first approach for simplicity:

In [7]:
for ft in ["model", "transmission", "fuelType"]: # Features you want to replace
    # Set each feature to the custom encoder
    json_ai.features[ft].encoder['module'] = 'LabelEncoder.LabelEncoder'

### 5) Generate code and your predictor from JSON-AI

Now, let's use this JSON-AI object to generate code and make a predictor. This can be done in 2 simple lines, below:

In [8]:
#Generate python code that fills in your pipeline
code = code_from_json_ai(json_ai)

# Turn the code above into a predictor object
predictor = predictor_from_code(code)

INFO:lightwood-2140:Unable to import black formatter, predictor code might be a bit ugly.


Now, let's run our pipeline. To do so, let's first:

(1) Perform a statistical analysis on the data (*this is important in preparing Encoders/Mixers as it populates the* `StatisticalAnalysis` *attribute with details some encoders need*). <br>
(2) Clean our data <br>
(3) Prepare the encoders <br>
(4) Featurize the data <br>

In [9]:
# Perform Stats Analysis
predictor.analyze_data(df)

# Pre-process the data
cleaned_data = predictor.preprocess(data=df)

# Create a train/test split
split_data = predictor.split(cleaned_data)

# Prepare the encoders 
predictor.prepare(split_data)

# Featurize the data
ft_data = predictor.featurize(split_data)

INFO:lightwood-2140:Performing statistical analysis on data
INFO:lightwood-2140:Starting statistical analysis
INFO:lightwood-2140:Finished statistical analysis
INFO:lightwood-2140:Cleaning the data
INFO:lightwood-2140:Splitting the data into train/test
INFO:lightwood-2140:Preparing the encoders
INFO:lightwood-2140:Encoder prepping dict length of: 1
INFO:lightwood-2140:Encoder prepping dict length of: 2
INFO:lightwood-2140:Encoder prepping dict length of: 3
INFO:lightwood-2140:Encoder prepping dict length of: 4
INFO:lightwood-2140:Encoder prepping dict length of: 5
INFO:lightwood-2140:Encoder prepping dict length of: 6
INFO:lightwood-2140:Encoder prepping dict length of: 7
INFO:lightwood-2140:Encoder prepping dict length of: 8
INFO:lightwood-2140:Encoder prepping dict length of: 9
INFO:lightwood-2140:Done running for: price
INFO:lightwood-2140:Categories Detected = 1
INFO:lightwood-2140:Done running for: model
INFO:lightwood-2140:Done running for: year
INFO:lightwood-2140:Categories Det

The splitter creates 3 data-splits, a "train", "dev", and "test" set. The `featurize` command from the predictor allows us to convert the cleaned data into features. We can access this as follows:

In [10]:
# Pick a categorical column name
col_name = "fuelType"

# Get the encoded feature data
enc_ft = ft_data["train"].get_encoded_column_data(col_name).squeeze(1) #torch tensor (N_rows x N_output_dim)

# Get the original data from the dataset
orig_data = ft_data["train"].get_column_original_data(col_name) #pandas dataframe

# Create a pandas data frame to compare encoded data and original data
compare_data = pd.concat([orig_data, pd.Series(enc_ft, name="EncData")], axis=1)
compare_data.head()

,fuelType,EncData
0,Diesel,1
1,Diesel,1
2,Diesel,1
3,Petrol,2
4,Diesel,1


We can see what the label mapping is by inspecting our encoders as follows:

In [11]:
# Label Name -> Label Number
print(predictor.encoders[col_name].label_dict)

{'Unknown': 0, 'Diesel': 1, 'Petrol': 2, 'Hybrid': 3}


For each category above, the number associated in the dictionary is the label for each category. This means "Diesel" is always represented by a 1, etc.

With that, you've created your own custom Encoder that uses a rule-based approach! Please checkout more [tutorials](https://lightwood.io/tutorials.html) for other custom approach guides.